#  causality sentence Identifier

## 1.  load sentence

In [17]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

pm_DB = pd.read_csv('data/in/KNN_RAG/SCITE_TrueFalse.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]

instances = pm_DB['sentence'].to_list()

len(instances)

786

## 2. load fewshot database

In [18]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()


In [19]:
semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. get causality label for sentence

In [20]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        window_size = len(p)

        # Slide through input_sentence with a window size of 8
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('p is ',p, 'i is', i)
                #print(f"Window '{window}' - Levenshtein Score: {score}")
    fewshot_index = list(set(fewshot_index))
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        #fewshot_index= fewshot_index[:10]
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [21]:

def get_fewshotexamples(fewshot_index):


    train_sentence = semeval_data['sentence'].to_list()

    fewshot_examples = [train_sentence[i] for i in fewshot_index]

    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    #fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples



In [22]:
import groq
from groq import Groq
client = groq.Groq(api_key= "enter your API key here..")


In [23]:
#gemma2-9b-it# llama3-8b-8192
def CS_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": """
            You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1; if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect)
          
         Below are around 20 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           """+ fewshot_examples 
            
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [24]:
import json
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

causality_identifier=[]
fewshotfound_inputsentence_index=[]
for i in range (len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    
    #KNN+RAG few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    
    fewshot_examples_rag = get_fewshotexamples(fewshot_index) #############fewshot
    #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    print('input_sentence : ', input_sentence)
    ###
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples_knn = [examples[j] for j in most_similar_indices]

   
    fewshot_comb = fewshot_examples_rag+fewshot_examples_knn
    fewshot_comb = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_comb)}
    fewshot_comb = json.dumps(fewshot_comb)   

    print('fewshot_comb : ', fewshot_comb)
    ################# KNN+RAG end---
   
    try:
        # output generation
        out = CS_Identifier(input_sentence, fewshot_comb)
        print('out', out)

    
        json_out = json.loads(out)
        causality_identifier_out = json_out['output']
        causality_identifier.append(int(causality_identifier_out))
        
    except TypeError as e:
        causality_identifier.append(0)
        continue
        
    except KeyError as e:
        causality_identifier.append(0)
        continue
    
    except groq.APIConnectionError as e:
        causality_identifier.append(0)
        continue
    except groq.RateLimitError as e:
        print('####################################ratelimit')
        causality_identifier.append(0)
        continue
    except groq.APIStatusError as e:
        causality_identifier.append(0)
        continue
    
    
    print(i)
    print(causality_identifier_out)
    
    

input_sentence :  This statement threw  people  into  chaos .
fewshot_comb :  {"example 1": "<effect>Anaphylaxis</effect> to <cause>calcitonin</cause>.", "example 2": "The most commonly recognized toxic effect of <cause>ethambutol</cause> is <effect>optic neuropathy</effect>, which generally is considered uncommon and reversible in medical literature.", "example 3": "<effect>Dermatitis</effect> to <cause>captopril</cause>.", "example 4": "The major side effect of <cause>infliximab</cause> is <effect>infection</effect>.", "example 5": "Allergic and <effect>irritant contact dermatitis</effect> to <cause>calcipotriol</cause>.", "example 6": "<effect>Systemic allergic contact dermatitis</effect> to <cause>8-methoxypsoralen</cause> (8-MOP).", "example 7": "CONCLUSIONS: The use of fluorouracil treatment with careful monitoring can be considered in a patient with mild <effect>allergic reactions</effect> to <cause>capecitabine</cause>.", "example 8": "After nine previous uncomplicated cycles s

input_sentence :   Paralysis  or  convulsions  are caused by <e3>hormone deficiencies and imbalances</e3>.
fewshot_comb :  {"example 1": "<cause>Quetiapine</cause> and <effect>obsessive-compulsive symptoms</effect> (OCS): case report and review of atypical antipsychotic-induced OCS.", "example 2": "\"<effect>Calluses</effect> are caused by improperly fitting shoes or by a <cause>skin abnormality</cause>.\"", "example 3": "<effect>Scleromyxedema</effect> in a patient with multiple sclerosis and monoclonal gammopathy on <cause>interferon beta-1a</cause>.", "example 4": "\"<cause>Frustrations</cause>, threats, and conflicts cause <effect>stress</effect>.\"", "example 5": "\"Sixty five million years ago an <cause>asteroid</cause> caused the <effect>extinction</effect> of the dinosaurs.\"", "example 6": "\"The <cause>compromises</cause> contained in the agreement caused the civil <effect>war</effect> in the 26 counties in June 1922-April 1923, in which the pro-Treaty Provisional Government 

input_sentence :  A qualified medical  professional  uses a variety of diagnostic  tests .
fewshot_comb :  {"example 1": "\"We continue to see progress toward a world free of the daily <effect>terror</effect> of antipersonnel <cause>landmines</cause>.\"", "example 2": "\"Villagers live in constant <effect>terror</effect> of indiscriminate <cause>gunfire</cause> which fills the nights throughout Chechnya.\"", "example 3": "PURPOSE: We report an unusual <effect>paradoxical effect</effect> of <cause>brimonidine</cause>.", "example 4": "We present an AIDS patient with severe and prolonged <effect>lactic acidosis</effect> on stavudine and <cause>lamivudine</cause>.", "example 5": "\"The basic <effect>indicator</effect> of the health <cause>condition</cause> of trees, stands and forests is tree defoliation.\"", "example 6": "<effect>Epstein-Barr virus-associated lymphoproliferative disorder</effect> in a patient with rheumatoid arthritis on <cause>methotrexate</cause> and rofecoxib: idiosync

input_sentence :  A polypropylene net, attached to the bottom of the fluidized bed section, acted as a flow distributor and a support for the static  bed  of  oysters .
fewshot_comb :  {"example 1": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>clozapine</cause> monotherapy.", "example 2": "<cause>Quetiapine</cause> and <effect>obsessive-compulsive symptoms</effect> (OCS): case report and review of atypical antipsychotic-induced OCS.", "example 3": "<effect>Fixed drug eruption</effect> to <cause>rofecoxib</cause>.", "example 4": "<effect>Lichenoid drug eruption</effect> to <cause>salsalate</cause>.", "example 5": "<effect>Transient asymptomatic bradycardia</effect> in patients on infusional <cause>5-fluorouracil</cause>.", "example 6": "<effect>Neurotoxicity</effect> of intrathecal <cause>methotrexate</cause>: MR imaging findings.", "example 7": "We present an AIDS patient with severe and prolonged <effect>lactic acid

input_sentence :  In the run-up to this match, a new  war  of words has erupted on the  internet  in yet another twist to the long-simmering saga.
fewshot_comb :  {"example 1": "PURPOSE: We report an unusual <effect>paradoxical effect</effect> of <cause>brimonidine</cause>.", "example 2": "The major side effect of <cause>infliximab</cause> is <effect>infection</effect>.", "example 3": "CONCLUSIONS: The use of fluorouracil treatment with careful monitoring can be considered in a patient with mild <effect>allergic reactions</effect> to <cause>capecitabine</cause>.", "example 4": "<effect>Neurotoxicity</effect> of intrathecal <cause>methotrexate</cause>: MR imaging findings.", "example 5": "\"The basic <effect>indicator</effect> of the health <cause>condition</cause> of trees, stands and forests is tree defoliation.\"", "example 6": "<effect>Fixed drug eruption</effect> to <cause>rofecoxib</cause>.", "example 7": "\"The <effect>singing</effect> comes from the substantial <cause>figure</ca

...

out {"output": 0}
773
0
input_sentence :  The citizens remained in their houses, and only vagabonds, the  scum  of various  nationalities , and spies were about; women were not to be seen, or only such as belonged to the dregs of society.
fewshot_comb :  {"example 1": "<effect>Scleromyxedema</effect> in a patient with multiple sclerosis and monoclonal gammopathy on <cause>interferon beta-1a</cause>.", "example 2": "This is the second report of <effect>lactic acidosis</effect> in a patient on stavudine and <cause>lamivudine</cause>.", "example 3": "\"The <effect>singing</effect> comes from the substantial <cause>figure</cause> of Mole - Paul Smith in the normal world - who crams into his days a full-time job.\"", "example 4": "\"The basic <effect>indicator</effect> of the health <cause>condition</cause> of trees, stands and forests is tree defoliation.\"", "example 5": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>cloz

out {"output": 1}
776
1
input_sentence :  So, the theme of the  play  has to do with  religion .
fewshot_comb :  {"example 1": "The patient suffered a <effect>life-threatening anaphylactoid reaction</effect> to <cause>amifostine</cause>.", "example 2": "<effect>Severe Raynaud's phenomenon</effect> with <cause>yohimbine</cause> therapy for erectile dysfunction.", "example 3": "<effect>Lichenoid drug eruption</effect> to <cause>salsalate</cause>.", "example 4": "<effect>Epstein-Barr virus-associated lymphoproliferative disorder</effect> in a patient with rheumatoid arthritis on <cause>methotrexate</cause> and rofecoxib: idiosyncratic reaction or pharmacogenetics?", "example 5": "We conclude <effect>peripheral neuropathy</effect> with <cause>5-FU</cause> is rare.", "example 6": "\"The randomized hypertensive ABCD study demonstrated a decrease in myocardial infarction with ACE inhibitors and a <effect>decrease</effect> in overall mortality with aggressive blood pressure <cause>control</cau

out {"output": 1}
779
1
input_sentence :  Information about Salmonellosis,  an illness  caused by  a bacteria  found in raw food, soil, or water.
fewshot_comb :  {"example 1": "\"<effect>Poverty</effect> is caused by <cause>overpopulation</cause>.\"", "example 2": "The most commonly recognized toxic effect of <cause>ethambutol</cause> is <effect>optic neuropathy</effect>, which generally is considered uncommon and reversible in medical literature.", "example 3": "\"Amtrak expects a normal rush hour after a fatal <cause>accident</cause> near Baltimore caused <effect>delays</effect> along the Northeast corridor.\"", "example 4": "Argatroban for <cause>heparin</cause>-induced <effect>thrombocytopenia</effect> in hepato-renal failure and CVVHD.", "example 5": "\"Before doing for his hair <effect>reduction</effect> by laser <cause>treatment</cause>, he first consulted with a dermatologist.\"", "example 6": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed wit

out {"output": 0}
782
0
input_sentence :  One of the most vivid portraits is that of Brookie Simmons, a white woman who fought to abolish child labor in a  town   factory .
fewshot_comb :  {"example 1": "The major side effect of <cause>infliximab</cause> is <effect>infection</effect>.", "example 2": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>clozapine</cause> monotherapy.", "example 3": "\"We continue to see progress toward a world free of the daily <effect>terror</effect> of antipersonnel <cause>landmines</cause>.\"", "example 4": "The most commonly recognized toxic effect of <cause>ethambutol</cause> is <effect>optic neuropathy</effect>, which generally is considered uncommon and reversible in medical literature.", "example 5": "Physicians should be cognizant of this potential complication in patients receiving <cause>thalidomide</cause> or <cause>thalidomide</cause>-like drugs who present with <effect>fever</eff

out {"output": 0}
785
0


In [25]:
len(causality_identifier)

786

In [26]:
len(causality_identifier)

786

In [27]:
len(fewshotfound_inputsentence_index)

0

In [28]:
#get  pattern_matched label
pattern_matched = [0]*(len(pm_DB))

# Update values to 1 at specified indexes
for index in fewshotfound_inputsentence_index:
    if index < len(pattern_matched):
        pattern_matched[index] = 1
pattern_matched

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [29]:
pm_DB['GPT_causality_label'] = causality_identifier
pm_DB['patternmatcher_label'] = pattern_matched
pm_DB

,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label
0,0,0,This statement threw people into chaos .,0,"[-0.0067518032155931, -0.003759827697649598, -...",0,0
1,1,1,Cutler ended up with a bleeding stomach ulcer...,1,"[-0.003396878018975258, -0.009281124919652939,...",0,0
2,2,2,This theme has been covered in science fict...,0,"[0.03351648524403572, -0.02911864034831524, 0....",0,0
3,3,3,Paralysis or convulsions are caused by <e3...,1,"[-0.023387493565678596, -0.02253008261322975, ...",0,0
4,4,4,"In economic terms, the ecological catastrophe...",1,"[0.008494839072227478, -0.038574811071157455, ...",0,0
...,...,...,...,...,...,...,...
781,781,781,"Before introducing Via in three test markets, ...",0,"[0.0033979706931859255, -0.009512247517704964,...",0,0
782,782,782,A programmer uses a high level language to...,0,"[0.011056014336645603, 0.002396157244220376, -...",0,0
783,783,783,One of the most vivid portraits is that of Bro...,0,"[-0.02381613664329052, -0.018164167180657387, ...",0,0
784,784,784,The earthquake caused the failures of the ...,1,"[-0.025205399841070175, 0.004254561383277178, ...",1,0


In [30]:
# Function to set 'prediction' column based on conditions
def set_prediction(row):
    if row['GPT_causality_label'] == 1 and row['patternmatcher_label'] == 1:
        return 1
    else:
        return 0

# Apply the function to each row and create the 'prediction' column
pm_DB['prediction'] = pm_DB.apply(set_prediction, axis=1)
pm_DB

,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label,prediction
0,0,0,This statement threw people into chaos .,0,"[-0.0067518032155931, -0.003759827697649598, -...",0,0,0
1,1,1,Cutler ended up with a bleeding stomach ulcer...,1,"[-0.003396878018975258, -0.009281124919652939,...",0,0,0
2,2,2,This theme has been covered in science fict...,0,"[0.03351648524403572, -0.02911864034831524, 0....",0,0,0
3,3,3,Paralysis or convulsions are caused by <e3...,1,"[-0.023387493565678596, -0.02253008261322975, ...",0,0,0
4,4,4,"In economic terms, the ecological catastrophe...",1,"[0.008494839072227478, -0.038574811071157455, ...",0,0,0
...,...,...,...,...,...,...,...,...
781,781,781,"Before introducing Via in three test markets, ...",0,"[0.0033979706931859255, -0.009512247517704964,...",0,0,0
782,782,782,A programmer uses a high level language to...,0,"[0.011056014336645603, 0.002396157244220376, -...",0,0,0
783,783,783,One of the most vivid portraits is that of Bro...,0,"[-0.02381613664329052, -0.018164167180657387, ...",0,0,0
784,784,784,The earthquake caused the failures of the ...,1,"[-0.025205399841070175, 0.004254561383277178, ...",1,0,0


In [31]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.7353689567430025
F1 Score: 0.5162790697674419
Precision: 0.46443514644351463
Recall: 0.581151832460733


In [16]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [ ]:
#pm_DB.to_csv('data/out/RAG_causality/task1/gemma/zero_SCITE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem